Notebook 1: Data Cleaning
=========================
The purpose of this notebook is to
1. Import the data
2. Clean the data
3. Export the data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import utils

# Import Dataset

In [27]:
import os
os.getcwd()
import importlib
importlib.reload(utils)

<module 'utils' from 'd:\\Documents\\GitHub\\road-accidents-fr\\code\\utils.py'>

In [28]:
dfd = utils.read_csv_of_year()

In [29]:
dfd['characteristics']

,Num_Acc,an,mois,jour,hrmn,lum,agg,int,atm,col,com,adr,gps,lat,long,dep
0,200500000001,5,1,12,1900,3,2,1,1.0,3.0,11.0,CD41B,M,5051500.0,294400.0,590
1,200500000002,5,1,21,1600,1,2,1,1.0,1.0,51.0,rue de Lille,M,5053700.0,280200.0,590
2,200500000003,5,1,21,1845,3,1,1,2.0,1.0,51.0,NaN,M,5054600.0,280000.0,590
3,200500000004,5,1,4,1615,1,1,1,1.0,5.0,82.0,NaN,M,5098700.0,240800.0,590
4,200500000005,5,1,10,1945,3,1,1,3.0,6.0,478.0,NaN,M,5096400.0,247500.0,590
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1121566,202100056514,2021,1,1,06:10,3,1,1,5.0,6.0,33021,NaN,NaN,"44,5266190000","-0,1955440000",33
1121567,202100056515,2021,1,1,10:20,1,1,1,2.0,6.0,38405,D520,NaN,"45,3802910000","5,7049140000",38
1121568,202100056516,2021,1,1,18:00,3,1,1,2.0,1.0,26064,Route dÃ©partementale 538,NaN,"44,9112100000","5,0196360000",26
1121569,202100056517,2021,1,1,10:55,1,1,2,1.0,6.0,33003,Voie rapide Bassens Ambes,NaN,"44,9542747363","-0,5179211363",33


# Clean Characteristics Dataset

## Translation of French Variable Names

In [64]:
dfd['characteristics'].rename(columns={'an': 'year', 'mois':'month', 'jour': 'day', 'hrmn':'hhmm', 
                                'lum': 'daylight', 'agg': 'built-up_area', 'int':'intersection_category', 'atm': 'weather',
                                'col': 'collision_category', 'com': 'municipality', 'adr':'adress', 'gps': 'gps_origin', 'lat': 'latitude',
                                'long': 'longitude', 'dep': 'department'}, inplace=True)

## Time and Date-Related Variables

In [ ]:

# Fix inconsistent year format
dfd['characteristics']['year'].replace({5:2005, 6:2006, 7:2007, 8:2008, 9:2009, 10:2010, 11:2011,
                                        12:2012, 13:2013, 14:2014, 15:2015, 16:2016, 17:2017, 18:2018},
                                        inplace=True)

# Fix inconsistent time format
dfd['characteristics']['hhmm'] = dfd['characteristics']['hhmm'].apply(lambda s: str(s).replace(':',''))

dfd['characteristics']['hour'] = dfd['characteristics']['hhmm'].apply(lambda hhmm: hhmm[:-2])
dfd['characteristics']['hour'] = dfd['characteristics']['hour'].replace('', np.nan) \
                                                               .fillna(method='bfill')
dfd['characteristics']['minute'] = dfd['characteristics']['hhmm'].apply(lambda hhmm: hhmm[-2:])

dfd['characteristics']['date'] = pd.to_datetime({'year':dfd['characteristics']['year'],
                                                 'month':dfd['characteristics']['month'],
                                                 'day':dfd['characteristics']['day'],
                                                 'hour':dfd['characteristics']['hour'],
                                                 'minute':dfd['characteristics']['minute']})
                                        
# New variable: weekday, integer from 0 to 6 representing the weekdays from monday to sunday.
dfd['characteristics']['day_of_week'] = dfd['characteristics']['date'].apply(lambda x: x.day_of_week)

# New binary variable: is_weekend, 0 for monday to friday and 1 for saturday and sunday
dfd['characteristics']['is_weekend'] = (dfd['characteristics']['day_of_week'] > 4).astype('int')


## Department Variable

In [62]:
def department_converter(dep):
    '''
    Takes in a department code as int and returns a string
    e.g. 750 will be '75' for Paris and 201 will be '2B'
    '''
    if dep == 201:
        return '2A'
    elif dep == 202:
        return '2B'
    elif dep>970:
        return str(dep)
    else:
        return str(dep).strip('0')

dfd['characteristics'].loc[(np.less(dfd['characteristics']['year'],2019)),'department'] = \
    dfd['characteristics'][(np.less(dfd['characteristics']['year'],2019))]['department'].apply(department_converter)

## Other 

In [ ]:
dfd['characteristics']['weather'] = dfd['characteristics']['weather'].fillna(
    dfd['characteristics']['weather'].mode()[0])
dfd['characteristics']['weather'].replace({-1, 0}, inplace=True)
dfd['characteristics']['weather'].astype('int')

In [ ]:
dfd['characteristics']['collision_category'] = dfd['characteristics']['collision_category'].fillna(
    dfd['characteristics']['collision_category'].mode()[0])

In [ ]:
dfd['characteristics']['built-up_area'].replace({1:0, 2:1}, inplace=True)                                

# Clean Locations Dataset

In [ ]:
# TODO

# Clean Vehicles Dataset

In [ ]:
# TODO

# Clean Persons Dataset

In [ ]:
# TODO